In [1]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata
import itertools
from tqdm import tqdm
import networkx as nx
import pickle
import os

tqdm.pandas()

In [11]:
final_df = pd.read_feather("data/alc_concept_map.feather")
full_bio_cohort = pd.read_feather('data/alc_cohort_details.feather')

In [3]:
final_df

,person_id,C0032961,C0041296,C0151632,CHEMBL112570,CHEMBL1200830,CHEMBL1222250,CHEMBL1618018,CHEMBL2146121,CHEMBL2219425,...,UBERON:0001088,UBERON:0001089,UBERON:0001135,UBERON:0001264,UBERON:0001690,UBERON:0001968,UBERON:0001969,UBERON:0001970,UBERON:0001988,UBERON:0002113
0,1242,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2615,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3423,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4681,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5806,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24946,10497163,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24947,10502015,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24948,10508290,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24949,10530139,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
spoke = np.load('../../psev_repo/PSEV_matrix')
sep = np.load('../../psev_repo/PSEV_SEP_map')
spoke_node = np.load('../../psev_repo/PSEV_SPOKE_node_map')

spoke = pd.DataFrame(spoke, columns=spoke_node)
spoke.index = sep
spoke.index = spoke.index.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
spoke.columns = spoke.columns.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

KeyboardInterrupt: 

In [5]:
spoke

,1,10,100,1000,10000,100008586,10001,10002,10003,100037417,...,X6RAB3,X6RAL5,X6RAN8,X6RAY8,X6RB12,X6REB3,X6REH9,X6RGC9,X6RLR1,X6RLX0
D010149,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
C0030201,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
C0007117,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,...,0.000003,0.000004,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
DOID:2513,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003,...,0.000003,0.000004,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003
C0022408,0.000003,0.000003,0.000003,0.000003,0.000003,0.000003,0.000002,0.000002,0.000002,0.000003,...,0.000002,0.000002,0.000003,0.000002,0.000003,0.000003,0.000002,0.000003,0.000003,0.000003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DB09297,0.000003,0.000003,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000003,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
DB01176,0.000003,0.000002,0.000003,0.000003,0.000003,0.000003,0.000003,0.000004,0.000003,0.000003,...,0.000003,0.000003,0.000003,0.000003,0.000003,0.000004,0.000004,0.000003,0.000003,0.000003
DB00995,0.000002,0.000003,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002
DB00726,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002


# Create Embeddings

In [6]:
def chunk_dot_product(df, matrix, num_chunks):
    results = []
    chunk_size = (df.shape[0] + num_chunks - 1) // num_chunks  # Calculate chunk size based on num_chunks
    
    for i in range(num_chunks):
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, df.shape[0])
        
        chunk = df.iloc[start_idx:end_idx]
        result_chunk = np.dot(chunk.values, matrix.values)
        results.append(result_chunk)
    
    return np.vstack(results)

In [12]:
person_id_index = final_df['person_id'].to_list()
final_df.drop(columns='person_id', inplace=True)

#Remove SEPs that are not in our final_df
spoke = spoke.loc[spoke.index.isin(final_df.columns)]

#Reindex final_df for dot product
final_df = final_df[spoke.index]

#PSEVs for each patient
patient_psevs = np.dot(final_df.values, spoke.values)

NameError: name 'spoke' is not defined

In [24]:
patient_psevs.shape
np.save('data/alc_unprocessed_psevs.npy', patient_psevs)

# Process the PSEVs

## Option 1: Top 30%, unsegmented on node type

In [11]:
# Step 1: Calculate variance for each column— do it in chunks bec memory
chunk_size = 1000
num_columns = patient_psevs.shape[1]
variances = []

for start in range(0, num_columns, chunk_size):
    end = min(start + chunk_size, num_columns)
    chunk = patient_psevs[:, start:end]
    chunk_variances = np.var(chunk, axis=0)
    variances.extend(chunk_variances)

variances = np.array(variances)

# Step 2: Determine the threshold for the top 30%
threshold = np.percentile(variances, 70)  # 70th percentile

# Step 3: Find the columns with variance above the threshold
selected_columns = variances > threshold

# Step 4: Filter the array to retain only these columns
filtered_patient_psevs = patient_psevs[:, selected_columns]

In [12]:
filtered_patient_psevs.shape

(24951, 116357)

In [16]:
np.save('data/alc_psevs/person_id_index.npy', np.array(person_id_index))
np.save('data/alc_psevs/filtered_patient_psevs.npy', filtered_patient_psevs)
np.save('data/alc_psevs/filtered_patient_psevs_columns.npy', np.array(spoke.columns[selected_columns]))

## Option 2: Node type segmented top 30% variance

In [ ]:
# spoke_columns = np.load('../../psev_repo/PSEV_SPOKE_node_map')
# spoke_columns = np.array([x.decode('utf-8') if isinstance(x, bytes) else x for x in spoke_columns])

In [ ]:
patient_psevs = np.load('data/alc_unprocessed_psevs.npy')

node_type = np.load('../../psev_repo/node_type_list.npy')
node_type = [x.decode('utf-8') if isinstance(x, bytes) else x for x in node_type]
node_type = pd.DataFrame({
    'node': spoke.columns,
    'type': node_type
})

unique_node_types = node_type['type'].unique()

In [ ]:
for nt in unique_node_types:
    nt_patient_psevs = patient_psevs[:, node_type[node_type['type'] == nt].index]
    nodes = node_type[node_type['type'] == nt]['node']

    # Step 1: Calculate variance for each column
    chunk_size = 1000
    num_columns = nt_patient_psevs.shape[1]
    variances = []

    for start in range(0, num_columns, chunk_size):
        end = min(start + chunk_size, num_columns)
        chunk = nt_patient_psevs[:, start:end]
        chunk_variances = np.var(chunk, axis=0)
        variances.extend(chunk_variances)

    variances = np.array(variances)

    # Step 2: Determine the threshold for the top 30%
    threshold = np.percentile(variances, 70)  # 70th percentile

    # Step 3: Find the columns with variance above the threshold
    selected_columns = variances > threshold

    # Step 4: Filter the array to retain only these columns
    filtered_array = nt_patient_psevs[:, selected_columns]
    nodes = np.array(nodes[selected_columns])

    #Save the node-specific file
    np.save(f'data/alc_nt_psevs/filtered_patient_psevs_{nt}.npy', filtered_array)
    np.save(f'data/alc_nt_psevs/filtered_patient_psevs_columns_{nt}.npy', nodes)

np.save('data/alc_nt_psevs/person_id_index.npy', np.array(person_id_index))


NameError: name 'person_id_index' is not defined